# 이미지에서 상품 속성 추출

이 노트북에서는 GPT-4o Vision을 활용하여 상품 이미지에서 자동으로 속성을 추출합니다.

### 주요 기능
- 🖼️ **이미지 분석**: GPT-4o Vision으로 상품 이미지 분석
- 📋 **속성 구조화**: OpenAI Structured Output으로 JSON 포맷 보장

## 0. 필요한 라이브러리 설치

In [1]:
##############################################
# 0. 라이브러리 설치 (최초 1회 실행)
##############################################
%pip install --quiet python-dotenv openai pillow requests azure-search-documents azure-identity pydantic

Note: you may need to restart the kernel to use updated packages.


## 1. 환경 변수 설정

In [2]:
import os
from dotenv import load_dotenv

##############################################
# 1. 환경 변수 로드 및 확인
##############################################
load_dotenv()

# AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION", "2024-10-21")
AZURE_OPENAI_ENDPOINT  = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_VISION_DEPLOYMENT_NAME  = os.getenv("AZURE_OPENAI_VISION_DEPLOYMENT_NAME")

AZURE_SEARCH_SERVICE_ENDPOINT = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
AZURE_SEARCH_INDEX_NAME = os.getenv("AZURE_SEARCH_INDEX_NAME")

print(f"OpenAI Api Version: {AZURE_OPENAI_API_VERSION}")
print(f"OpenAI Endpoint: {AZURE_OPENAI_ENDPOINT}")
print(f"OpenAI Vision Deployment: {AZURE_OPENAI_VISION_DEPLOYMENT_NAME}")
print(f"Search Endpoint: {AZURE_SEARCH_SERVICE_ENDPOINT}")
print(f"Search Index: {AZURE_SEARCH_INDEX_NAME}")

OpenAI Api Version: 2024-10-21
OpenAI Endpoint: https://open-ai-seoulcentral-1023.openai.azure.com/
OpenAI Vision Deployment: gpt-4o
Search Endpoint: https://ai-search-commercial-1023.search.windows.net
Search Index: products-index


## 2. 초기화 및 환경 설정

In [ ]:
import os
import base64
import json
from openai import AzureOpenAI
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
import requests
import os, shutil

##############################################
# 2-0. Azure 로그인을 위한 환경변수 등록 (az cli 경로 못찾는 경우)
##############################################
# AZ_DIR = "/opt/homebrew/bin" # 변경 필요
# os.environ["PATH"] = f"{AZ_DIR}:" + os.environ["PATH"]
# print("az which:", shutil.which("az"))


# Azure OpenAI 서비스 인증 설정 (임베딩용)
credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(
    credential,
    "https://cognitiveservices.azure.com/.default"
)

# Azure OpenAI 클라이언트 초기화
client = AzureOpenAI(
    api_version=AZURE_OPENAI_API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    azure_ad_token_provider=token_provider
)

az which: /opt/homebrew/bin/az


## 3. 기본 이미지 분석 테스트

먼저 간단한 이미지 분석을 테스트해봅시다.

In [4]:
# 간단한 설명 요청
simple_prompt = "이 이미지에 있는 상품에 대해서 고객이 검색해보기 편하도록 자세하게 한글로 설명해주세요."
sample_image_url = "https://images.unsplash.com/photo-1525966222134-fcfa99b8ae77?w=800&q=80"

response = client.chat.completions.create(
    model=AZURE_OPENAI_VISION_DEPLOYMENT_NAME,
    messages=[
        {"role": "system", "content": "상품에 대한 정보를 한글로 추출합니다."},
        {
            "role": "user",
            "content": [
                {"type": "text", "text": simple_prompt},
                {
                    "type": "image_url",
                    "image_url": {"url": sample_image_url},
                }
            ]
        }
    ],
    max_tokens=2048
)

result = response.choices[0].message.content

print("🖼️ 이미지 분석 결과:")
print("="*80)
print(result)
print("="*80)

🖼️ 이미지 분석 결과:
이 이미지는 클래식한 스타일의 스니커즈를 보여줍니다. 신발은 와인색(버건디) 캔버스 소재로 제작되었으며, 측면에는 흰색 곡선형 라인 디테일이 돋보입니다. 끈으로 고정할 수 있는 레이스업 디자인이고, 밑창은 내구성이 강한 흰색 고무 소재로 되어 있습니다. 캐주얼하면서도 스타일리시한 느낌을 주며, 일상복부터 스트릿 패션까지 다양한 스타일링에 잘 어울릴 수 있는 디자인을 제공합니다.

검색어 추천:
- 와인색 스니커즈
- 클래식 캔버스 신발
- 흰색 곡선 디테일 스니커즈
- 레이스업 디자인 캐주얼화
- 스트릿 패션 스니커즈


## 4. 구조화된 상품 속성 추출

이제 실제 이커머스에서 필요한 구조화된 데이터를 추출해봅시다.

In [5]:
# 간단한 설명 요청
simple_prompt = "이 이미지에 있는 상품에 대해서 고객이 검색해보기 편하도록 자세하게 한글로 설명해주세요."
sample_image_url = "https://images.unsplash.com/photo-1525966222134-fcfa99b8ae77?w=800&q=80"

# JSON Schema 정의
schema = {
    "type": "object",
    "properties": {
        "imageCaption": {"type": "string", "description": "한국어 캡션"},
        "imageDescription": {"type": "string", "description": "한국어 자세한 설명"},
        "imageTags": {
            "type": "array",
            "items": {"type": "string"},
            "description": "핵심 태그 목록"
        }
    },
    "required": ["imageCaption", "imageDescription", "imageTags"],
    "additionalProperties": False
}
    
response = client.chat.completions.create(
    model=AZURE_OPENAI_VISION_DEPLOYMENT_NAME,
    messages=[
        {"role": "system", "content": "상품에 대한 정보를 한글로 추출합니다."},
        {
            "role": "user",
            "content": [
                {"type": "text", "text": simple_prompt},
                {
                    "type": "image_url",
                    "image_url": {"url": sample_image_url},
                }
            ]
        }
    ],
    # JSON Schema 포맷 지시
    response_format={
        "type": "json_schema",
        "json_schema": {"name": "ImageCaptionSchema", "schema": schema}
    },
    max_tokens=2048
)

result = response.choices[0].message.content

print("🖼️ 이미지 분석 구조화된 결과:")
print("="*80)
result_json = json.loads(result)
print(json.dumps(result_json, indent=2, ensure_ascii=False))
print("="*80)

🖼️ 이미지 분석 구조화된 결과:
{
  "imageCaption": "조화로운 색상이 돋보이는 스니커즈 한 켤레.",
  "imageDescription": "이 이미지는 밝은 주황색 배경에서 마룬(자주색)과 흰색이 조화를 이룬 스니커즈를 보여줍니다. 스니커즈는 매끈한 디자인으로, 스타일리시하면서도 캐주얼한 패션에 잘 어울립니다. 하얀색 끈과 밑창은 자주색 부분과 대조를 이루며 모던한 분위기를 자아냅니다.",
  "imageTags": [
    "스니커즈",
    "운동화",
    "패션",
    "밝은",
    "캐주얼",
    "자주색",
    "패션 아이템"
  ]
}


In [6]:
import os
from pydantic import BaseModel

# 응답 포맷 지정
class ProductInfo(BaseModel):
    imageCaption: str
    imageDescription: str
    imageTags: list[str]

# 간단한 설명 요청
simple_prompt = "이 이미지에 있는 상품에 대해서 고객이 검색해보기 편하도록 자세하게 한글로 설명해주세요."
sample_image_url = "https://images.unsplash.com/photo-1525966222134-fcfa99b8ae77?w=800&q=80"
    
completion = client.chat.completions.parse(
    model=AZURE_OPENAI_VISION_DEPLOYMENT_NAME,
    messages=[
        {"role": "system", "content": "상품에 대한 정보를 한글로 추출합니다."},
        {
            "role": "user",
            "content": [
                {"type": "text", "text": simple_prompt},
                {
                    "type": "image_url",
                    "image_url": {"url": sample_image_url},
                }
            ]
        }
    ],
    # JSON Schema 포맷 지시
    response_format=ProductInfo,
    max_tokens=2048
)

event = completion.choices[0].message.parsed

print("🖼️ 이미지 분석 구조화된 결과:")
print("="*80)
print(event.imageCaption)
print(event.imageDescription)
print(event.imageTags)
print("="*80)

🖼️ 이미지 분석 구조화된 결과:
와인 레드 색상의 스니커즈
이 이미지는 와인 레드 색상의 스니커즈를 보여주며, 흰색 고무 밑창과 흰 끈, 측면에 흰 라인이 특징입니다.
['신발', '스니커즈', '패션', '와인레드', '흰색라인']


## 5. 기존 제품 정보를 확장


In [7]:
##############################################
# 5-1. 기존 제품 데이터 로드
##############################################
import json
from pathlib import Path

# 데이터 파일 경로
data_path = Path("data/sample_products.json")

# JSON 파일 읽기
with open(data_path, "r", encoding="utf-8") as f:
    products = json.load(f)

print(f"✅ 총 {len(products)}개의 제품 데이터를 로드했습니다.")
print(f"\n첫 번째 제품 예시:")
print(json.dumps(products[0], indent=2, ensure_ascii=False))

✅ 총 30개의 제품 데이터를 로드했습니다.

첫 번째 제품 예시:
{
  "id": "PROD001",
  "name": "클래식 화이트 티셔츠",
  "brand": "베이직코튼",
  "description": "100% 순면으로 제작된 기본 화이트 티셔츠입니다. 데일리룩으로 완벽하며 편안한 착용감을 제공합니다.",
  "imageUrl": "https://images.unsplash.com/photo-1521572163474-6864f9cf17ab?w=800&q=80",
  "price": 29000
}


In [8]:
##############################################
# 5-2. 이미지 분석 함수 정의
##############################################
from pydantic import BaseModel

class ProductInfo(BaseModel):
    imageCaption: str
    imageDescription: str
    imageTags: list[str]

def analyze_product_image(image_url: str, retry_count: int = 3) -> ProductInfo:
    """
    제품 이미지를 분석하여 캡션, 설명, 태그를 추출합니다.
    
    Args:
        image_url: 분석할 이미지 URL
        retry_count: 실패 시 재시도 횟수
        
    Returns:
        ProductInfo: 이미지 분석 결과
    """
    prompt = "이 이미지에 있는 상품에 대해서 고객이 검색해보기 편하도록 자세하게 한글로 설명해주세요."
    
    for attempt in range(retry_count):
        try:
            completion = client.chat.completions.parse(
                model=AZURE_OPENAI_VISION_DEPLOYMENT_NAME,
                messages=[
                    {"role": "system", "content": "상품에 대한 정보를 한글로 추출합니다."},
                    {
                        "role": "user",
                        "content": [
                            {"type": "text", "text": prompt},
                            {
                                "type": "image_url",
                                "image_url": {"url": image_url},
                            }
                        ]
                    }
                ],
                response_format=ProductInfo,
                max_tokens=2048
            )
            
            return completion.choices[0].message.parsed
            
        except Exception as e:
            if attempt < retry_count - 1:
                print(f"⚠️ 재시도 중... ({attempt + 1}/{retry_count})")
                continue
            else:
                print(f"❌ 이미지 분석 실패: {e}")
                raise
                
print("✅ 이미지 분석 함수가 정의되었습니다.")

✅ 이미지 분석 함수가 정의되었습니다.


In [9]:
##############################################
# 5-3. 모든 제품 이미지 분석 및 데이터 업데이트
##############################################
from time import sleep
from tqdm.notebook import tqdm

# 업데이트된 제품 데이터를 저장할 리스트
augmented_products = []

print(f"🚀 {len(products)}개의 제품 이미지를 분석합니다...\n")

# 각 제품의 이미지 분석
for idx, product in enumerate(tqdm(products, desc="제품 이미지 분석")):
    try:
        # 이미지 URL 확인
        image_url = product.get("imageUrl")
        
        if not image_url:
            print(f"⚠️ 제품 ID {product.get('id')}: 이미지 URL이 없습니다. 건너뜁니다.")
            augmented_products.append(product)
            continue
        
        # 이미지 분석
        print(f"\n📸 [{idx+1}/{len(products)}] {product.get('name', 'Unknown')} 분석 중...")
        result = analyze_product_image(image_url)
        
        # 기존 제품 데이터 복사 및 업데이트
        updated_product = product.copy()
        updated_product["imageCaption"] = result.imageCaption
        updated_product["imageDescription"] = result.imageDescription
        updated_product["imageTags"] = result.imageTags
        
        augmented_products.append(updated_product)
        
        print(f"   ✅ 캡션: {result.imageCaption[:50]}...")
        print(f"   ✅ 설명: {result.imageDescription[:50]}...")
        print(f"   ✅ 태그: {', '.join(result.imageTags[:5])}")
        
        # API Rate Limit 고려하여 짧은 대기
        sleep(0.5)
        
    except Exception as e:
        print(f"❌ 제품 ID {product.get('id')} 처리 중 오류: {e}")
        # 오류 발생 시 원본 데이터 유지
        augmented_products.append(product)
        continue

print(f"\n✅ 총 {len(augmented_products)}개의 제품 처리 완료!")

🚀 30개의 제품 이미지를 분석합니다...



제품 이미지 분석:   0%|          | 0/30 [00:00<?, ?it/s]


📸 [1/30] 클래식 화이트 티셔츠 분석 중...
   ✅ 캡션: A person wearing a plain white T-shirt with a back...
   ✅ 설명: 이미지는 심플한 디자인의 흰색 티셔츠를 입고 있는 사람을 보여줍니다. 배경에는 다양한 식물...
   ✅ 태그: 흰색, 티셔츠, 심플, 캐주얼, 패션

📸 [2/30] 슬림핏 청바지 분석 중...
   ✅ 캡션: 세 가지 종류의 데님과 블랙 진이 놓여 있습니다....
   ✅ 설명: 이미지에는 서로 다른 색상의 청바지 세 벌이 나란히 정렬되어 진열된 모습이 담겨 있습니다....
   ✅ 태그: 청바지, 블랙진, 로고, 패션, 의류

📸 [3/30] 플로럴 원피스 분석 중...
   ✅ 캡션: 붉은색 드레스를 입은 한 사람이 회전하며 사진을 찍고 있음....
   ✅ 설명: 사진 속 여성은 긴 붉은색 드레스를 입고 정원의 풍경 속에서 포즈를 취하고 있습니다. 드레...
   ✅ 태그: 드레스, 붉은색, 이브닝웨어, 패션, 여성 의류

📸 [4/30] 러닝 운동화 분석 중...
   ✅ 캡션: 붉은 색 나이키 러닝화...
   ✅ 설명: 나이키 스포츠 브랜드의 러닝화로, 빨간색 니트 소재로 제작되었으며 흰색 스우시 로고가 특징...
   ✅ 태그: 스포츠, 신발, 나이키, 러닝화, 빨간색

📸 [5/30] 가죽 크로스백 분석 중...
   ✅ 캡션: 주황색 톤의 고급스러운 가죽과 라탄 디자인의 숄더백....
   ✅ 설명: 이 제품은 주황색 가죽으로 마감된 상단 부분과 라탄으로 짠 하단 부분이 독특하게 조합된 핸...
   ✅ 태그: 숄더백, 라탄, 가죽, 주황색, 고급스러움

📸 [6/30] 오버핏 후드티 분석 중...
   ✅ 캡션: 외부에서 드론을 조종하며 회색 후드 티셔츠 착용....
   ✅ 설명: 이미지에는 회색 후드 티셔츠를 입은 사람이 풍경이 펼쳐진 배경에서 드론을 조종하는 모습이 ...
   ✅ 태그: 드론, 야외, 후드티, 풍경, 취미

📸 [7/30] 미니멀 손목시

In [10]:
##############################################
# 5-4. 업데이트된 데이터를 파일로 저장
##############################################

# 저장할 파일 경로
output_path = Path("data/sample_products_augmented.json")

# JSON 파일로 저장
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(augmented_products, f, ensure_ascii=False, indent=2)

print(f"✅ 업데이트된 제품 데이터가 저장되었습니다: {output_path}")
print(f"📊 총 {len(augmented_products)}개의 제품")

# 결과 샘플 출력
print(f"\n📋 업데이트된 첫 번째 제품 예시:")
print("="*80)
print(json.dumps(augmented_products[0], indent=2, ensure_ascii=False))
print("="*80)

✅ 업데이트된 제품 데이터가 저장되었습니다: data/sample_products_augmented.json
📊 총 30개의 제품

📋 업데이트된 첫 번째 제품 예시:
{
  "id": "PROD001",
  "name": "클래식 화이트 티셔츠",
  "brand": "베이직코튼",
  "description": "100% 순면으로 제작된 기본 화이트 티셔츠입니다. 데일리룩으로 완벽하며 편안한 착용감을 제공합니다.",
  "imageUrl": "https://images.unsplash.com/photo-1521572163474-6864f9cf17ab?w=800&q=80",
  "price": 29000,
  "imageCaption": "A person wearing a plain white T-shirt with a background of plants.",
  "imageDescription": "이미지는 심플한 디자인의 흰색 티셔츠를 입고 있는 사람을 보여줍니다. 배경에는 다양한 식물들이 놓여져 있는 공간이 보입니다.",
  "imageTags": [
    "흰색",
    "티셔츠",
    "심플",
    "캐주얼",
    "패션",
    "식물",
    "심미적"
  ]
}
